<a href="https://colab.research.google.com/github/fabiocastilhoss/StepAnalyzerPAT2Math/blob/main/v3_step_analyzer_pat2math.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step Analyser PAT2Math - Versão 3

*   Versão 3
*   Redes Neurais
*   ~ 115 mil instâncias
*   Com correção de rótulos

Instalação de pacotes e importações

In [ ]:
pip install accelerate -U

In [ ]:
import tensorflow as tf #Manipulação de expressões matemáticas sobre tensores numéricos
from tensorflow import keras #Biblioteca para Deep Learning
from keras import layers #Estruturas de dados para Deep Learning
from keras.preprocessing.sequence import pad_sequences #Pacote para preenchimento de sequências
from keras import activations, optimizers, losses #Módulos para ativação, otimizadores e funções de loss
from keras.callbacks import ModelCheckpoint #pacote para salvar o melhor modelo
from keras.preprocessing.text import Tokenizer #Tokenizador
from keras.models import Sequential #Classe para criação de redes neurais
from keras.layers import Embedding, GRU, Dense, Dropout #Pacote para criação de redes neurais
from keras.layers import Input, concatenate #Pacote para criação de redes neurais
from keras.models import Model #Pacote para criação de redes neurais
import matplotlib.pyplot as plt #Biblioteca para visualização de dados
import seaborn as sns #Biblioteca para visualização de dados
import numpy as np #Operações em arrays multidimensionais
import pandas as pd #Biblioteca para análise e manipulação de dados
import random #Geração de números aleatórios
import string #Operações em Strings
import re #Operações em expressões regulares
from sklearn.model_selection import train_test_split #Divisão de conjuntos
from sklearn.metrics import confusion_matrix #Matriz de confusão
from sklearn.metrics import f1_score, precision_score, recall_score #Importa F1, Precision e Recall
from sklearn.metrics import roc_curve, auc #Importa curva ROC
from sklearn.utils import shuffle #Embaralhamento
from sklearn.metrics import accuracy_score #Métricas
from google.colab import drive #Conexão do Google Colab ao Google Drive
import sympy as sp #Biblioteca para resolução de matemática simbólica

Leitura e filtragem do log das interações dos estudantes no PAT2Math.

In [ ]:
#Conexão com o Google Drive
drive.mount('/content/drive')

In [ ]:
#Leitura do log de interações do PAT2Math, armazenado no Google Drive
log = pd.read_csv("/content/drive/MyDrive/colab/fabio-mestrado/log_with_current_step_pat2math.csv", sep=",", encoding="latin-1")
log.head()

In [ ]:
#Filtragem do log para quatro colunas
log = log[["initial_equation", "last_correct_step", "currentStep", "step_is_correct"]]
log.head()

In [ ]:
# Verifica a quatidade de dados faltantes em cada coluna
dados_faltantes = log.isna().sum()
print(dados_faltantes)

# Junção de colunas:
Para ampliação do conjunto de dados, quando a coluna last_correct_step é diferente de initial_equation, ela é usada como equação inicial, ou seja, uma nova instância é criada, contendo os dados de last_correct_step na coluna initial_equation, mantendo os dados de currentStep e step_is_correct, nas suas colunas equivalentes.

No final do procedimento, a coluna last_correct_step é excluída.

In [ ]:
#Amplia o número de instâncias, inserindo last_correct_step como initial_equation, quando contiverem valores diferentes.

# Criar uma lista para armazenar os novos dados
new_rows = []

# Iterar sobre cada linha do DataFrame
for index, row in log.iterrows():
    if row['initial_equation'] != row['last_correct_step']:
        new_row = row.copy()  # Copiar a linha existente
        new_row['initial_equation'] = row['last_correct_step']  # Atualizar 'initial_equation'
        new_rows.append(new_row)  # Adicionar a nova linha à lista

# Adicionar as novas linhas ao DataFrame
log = log.append(new_rows, ignore_index=True)

In [ ]:
#Filtra o log resultante para apenas três colunas
log = log[["initial_equation", "currentStep", "step_is_correct"]]
log

A junção de colunas resulta em 251.777 instâncias.

# Tratamento de Dados:

*   Retirada de espaços em branco
*   Retiradas de duplicatas
*   Retirada de dados faltantes
*   Verificação de caracteres inválidos
*   Padronização da equação
*   Checagem e correção dos rótulos

## Retira espaços em branco

Retira espaços em branco nas colinas initial_equation e currentStep.

In [ ]:
#Exclui espaços em branco
log["initial_equation"] = log["initial_equation"].replace(" ", "")
log["currentStep"] = log["currentStep"].replace(" ", "")

## Retira duplicatas

Exclui instâncias que contém dados duplicados, usando como chave as duas colunas: initial_equation e currentStep. Só exclui quando ambas contiverem valores iguais, reduzindo o número de instâncias para 34.769.

In [ ]:
#Exclui instâncias duplicadas
log = log.drop_duplicates(subset=['initial_equation', 'currentStep'])
log.head(), log.shape

## Retira linhas com valores faltantes

Exclui as instâncias que contém valores faltantes, reduzindo o número de instâncias para 30.279.

In [ ]:
#Exclui as instâncias que contém valores faltantes
log = log.dropna()
log.head(), log.shape

Verifica o número de rótulos corretos e incorretos

In [ ]:
#Conta número de instâncias com label incorreto e correto
counts = log["step_is_correct"].value_counts()
counts

In [ ]:
# Crie um gráfico de barras
plt.bar(counts.index, counts.values, color=['red', 'blue'], edgecolor='k')

# Adicione rótulos e título
plt.xlabel("Status")
plt.ylabel("Instâncias")
plt.title("Contagem de Instâncias com Rótulo Incorreto e Correto")

# Personalize os rótulos no eixo x (opcional)
plt.xticks(counts.index, ['Incorreto', 'Correto'])

# Mostre o gráfico
plt.show()

## Correção de rótulos

Primeiramente, a função has_invalid_characters verifica se a equação possui caracteres inválidos e insere o resultado (Verdadeiro ou Falso) em uma nova coluna.

Após isso, a equação é padronizada e a função resolver_e_comparar resolve initial_equation e currentStep e verifica se seus resultados são iguais (indicando que o passo está correto e que o rótulo deveria constar como correto). Com base nisso, insere uma nova coluna informando se o rótulo do dataset está certo ou errado.

Por fim, a função altera_label, altera os rótulos que estiverem anotados de forma incorreta.

In [ ]:
#Função que verifica se a equação contém caracteres inválidos
def has_invalid_characters(text):

    # Use uma expressão regular para encontrar qualquer caractere que não esteja na lista permitida
    pattern = r'[^*+\-()/=xX\d\s]'  # Qualquer coisa que não seja *, +, -, /, (, ), =, x, dígito ou espaço em branco
    return bool(re.search(pattern, text))

# Aplica a função has_invalid_characters à coluna currentStep e cria uma nova coluna "has_invalid_chars"
log['has_invalid_chars'] = log['currentStep'].apply(has_invalid_characters)

# Exibe as linhas onde foram encontrados caracteres inválidos
invalid_rows = log[log['has_invalid_chars'] == True]
invalid_rows

In [ ]:
#Conta o número de instâncias que contém caracteres inválidos
log["has_invalid_chars"].value_counts()

Ao todo são 1.118 instâncias que contém caracteres inválidos.

In [ ]:
#Função para padronizar uma equação
def padronizar_equacao(eq_str):
    # Remove espaços em branco e transforma "X" em "x"
    eq_str = eq_str.replace(" ", "").replace("X", "x")

    # Adiciona "*" entre coeficiente e variável (por exemplo, 2x vira 2*x)
    eq_str = re.sub(r'(\d)([a-z])', r'\1*\2', eq_str)

    # Adiciona "*" entre coeficiente e abertura de parênteses (por exemplo, 2(2+4) vira 2*(2+4))
    eq_str = re.sub(r'(\d+)\(', r'\1*(', eq_str)

    # Adiciona "*" entre variável x e abertura de parênteses (por exemplo, x(2+4) vira x*(2+4))
    eq_str = re.sub(r'(x)(\()', r'\1*\2', eq_str)

    return eq_str

In [ ]:
#Função que acha a solução de duas equações e retorna 1 se as soluções são iguais (passo correto) e 0 se são diferentes (passo incorreto)
def resolver_e_comparar(eq1_str, eq2_str):

    # Indicação de variáveis em SymPy
    x = sp.symbols('x')

    try:
        # Padronizar o formato das equações
        eq1_str = padronizar_equacao(eq1_str)
        eq2_str = padronizar_equacao(eq2_str)

        # Retorna 0 (passo incorreto) se a segunda equação (currentStep) possui caracteres inválidos - Sabe-se que são 1.118 instâncias
        if has_invalid_characters(eq2_str):
            return 0

        # Retorna 0 (passo incorreto) se as equações estão vazias
        if not eq1_str or not eq2_str:
            return 0

        # Retorna 0 (passo incorreto) se as equações não possuem sinal de "="
        if "=" not in eq1_str or "=" not in eq2_str:
            return 0

        # Divide ambas as equações pelo sinal de "=" e cria duas listas com as partes (eq1_parts e eq2_parts)
        eq1_parts = eq1_str.split('=')
        eq2_parts = eq2_str.split('=')

        # Retorna 0 (passo incorreto) se as equações contém um número de partes diferente de 2
        if len(eq1_parts) != 2 or len(eq2_parts) != 2:
            return 0

        # Retorna 0 (passo incorreto) se as partes da equação 2 (currentStep) estão vazias
        if eq2_parts[0] == "" or eq2_parts[1] == "":
            return 0

        # Retira os espaços em branco das partes da equação 2 (currentStep)
        eq2_parts[0] = eq2_parts[0].strip()
        eq2_parts[1] = eq2_parts[1].strip()

        # Retorna 0 (passo incorreto) se as partes da equação 2 (currentStep) são iguais a "-0"
        if eq2_parts[0] == "-0" or eq2_parts[1] == "-0":
            return 0

        # Retorna 0 (passo incorreto) se as partes da equação 2 (currentStep) são iguais a "0/"
        if eq2_parts[0].find("0/") != -1:
            return 0
        if eq2_parts[1].find("0/") != -1:
            return 0

        # Retorna 0 (passo incorreto) se as partes da equação 2 (currentStep) são iguais a "(0)/"
        if eq2_parts[0].find("(0)/") != -1:
            return 0
        if eq2_parts[1].find("(0)/") != -1:
            return 0

        # Simplifica e formata as equações para uso no SymPy
        eq1 = sp.Eq(sp.sympify(eq1_parts[0]), sp.sympify(eq1_parts[1]))
        eq2 = sp.Eq(sp.sympify(eq2_parts[0]), sp.sympify(eq2_parts[1]))

        # Tentar resolver as equações apenas se elas estiverem bem formadas
        sol1 = sp.solve(eq1, x)
        sol2 = sp.solve(eq2, x)

        # Compara as soluções das duas equações, retornando 1 para soluções iguais (passo correto) e 0 para soluções diferentes (passo incorreto)
        if sol1 == sol2:
            return 1
        else:
            return 0
    # Lança exceção no caso de erros no SymPy e retorna 0 (passo incorreto)
    except (sp.SympifyError, ValueError, TypeError) as e:
        return 0

In [ ]:
# Aplica a função resolver_e_comparar a cada linha do DataFrame e cria a coluna label_verifica
log['label_verifica'] = log.apply(lambda row: resolver_e_comparar(row['initial_equation'], row['currentStep']), axis=1)

# Mapea os valores de "step_is_correct" e atualiza a coluna "label_verifica" (Label Correto ou Label Incorreto, de acordo com o resultado da função resolver_e_comparar)
log['label_verifica'] = log.apply(lambda row: "Label Correto" if (row['label_verifica'] == 1 and row['step_is_correct'] == 1)
                                           else ("Label Correto" if (row['label_verifica'] == 0 and row['step_is_correct'] == 0)
                                                else "Label Incorreto"), axis=1)

# Visualizar o DataFrame resultante
log

São 28.218 instâncias com rótulo anotado corretamente e 2.061 instâncias com rótulo anotado de forma incorreta

In [ ]:
log.value_counts("label_verifica")

In [ ]:
value_counts_series = log[log['label_verifica'] == 'Label Incorreto']['step_is_correct'].value_counts()

# Acesse as contagens específicas para 1.0 e 100
count_label_incorreto_step_1 = value_counts_series.get(1.0, 0)
count_label_incorreto_step_0 = value_counts_series.get(0.0, 0)

print(f"Quantidade de instâncias com 'Label Incorreto' e step_is_correct igual a 1.0: {count_label_incorreto_step_1}")
print(f"Quantidade de instâncias com 'Label Incorreto' e step_is_correct igual a 0.0: {count_label_incorreto_step_0}")

Desses 2.061 rótulos anotados incorretamente, 1.785 constam como incorretos e deveriam constar como corretos e 276 constam como corretos e deveriam constar como incorretos.

In [ ]:
#Função que altera os rótulos anotados incorretamente
def altera_label(step_is_correct, label_verifica):

    # Se o rótulo foi anotado incorretamente, inverte o seu valor (se é 1 vira 0 e se é 0 vira 1)
    if label_verifica == "Label Incorreto":
        if step_is_correct == 1.0:
            step_is_correct = 0.0
        elif step_is_correct == 0.0:
            step_is_correct = 1.0

    return step_is_correct

#Atualiza os rótulos com a função altera_label
log['step_is_correct'] = log.apply(lambda row: altera_label(row['step_is_correct'], row['label_verifica']), axis=1)

Após as correções dos rótulos, o código de verificação é novamente chamado para checar se a atualização funcionou corretamente.

In [ ]:
# Aplica a função resolver_e_comparar a cada linha do DataFrame e cria uma coluna label_verifica
log['label_verifica'] = log.apply(lambda row: resolver_e_comparar(row['initial_equation'], row['currentStep']), axis=1)

# Mapea os valores de "step_is_correct" e atualiza a coluna label_verifica
log['label_verifica'] = log.apply(lambda row: "Label Correto" if (row['label_verifica'] == 1 and row['step_is_correct'] == 1)
                                           else ("Label Correto" if (row['label_verifica'] == 0 and row['step_is_correct'] == 0)
                                                else "Label Incorreto"), axis=1)

In [ ]:
#Conta novamente o número de rótulos verificados como informados correta ou incorretamente
log.value_counts("label_verifica")

A nova verificação não encontrou valores de rótulos incorretos.

In [ ]:
#Conta novamente o número de rótulos corretos e incorretos
counts = log.value_counts("step_is_correct")
counts

Agora, são 20.919 rótulos incorretos e 9.630 corretos.

In [ ]:
# Crie um gráfico de barras
plt.bar(counts.index, counts.values, color=['red', 'blue'], edgecolor='k')

# Adicione rótulos e título
plt.xlabel("Status")
plt.ylabel("Instâncias")
plt.title("Contagem de Instâncias com Rótulo Incorreto e Correto")

# Personalize os rótulos no eixo x (opcional)
plt.xticks(counts.index, ['Incorreto', 'Correto'])

# Mostre o gráfico
plt.show()

In [ ]:
#Exclui as colunas label_verifica e has_invalid_chars
log = log[["initial_equation", "currentStep", "step_is_correct"]]

In [ ]:
#Imprime o log atualizado
log

# Pré-Processamento

*    Balanceamento de Dados - Data Augmentation

## Balanceamento de Dados

O balanceamento de dados é realizado através da função data augmentation.

*    A primeira função de data augmentation (augment_equation) recebe uma equação da coluna initial equation e o indicador correct (para gerar uma equação correta ou incorreta). Essa função, quando correct é informado como 1, insere operações aleatórias de adição, subtração, multiplicação e divisão dos dois lados da equação. No final, também aleatoriamente, pode inverter os lados da equação. Se correct é 0, a inserção é realizada em apenas um lado da equação, tornando-a incorreta.

*    A segunda função (augment_equation_current) recebe uma equação da coluna initial equation, uma de currentStep e o indicador correct (para gerar uma equação correta ou incorreta). Essa função, quando correct é 1, copia o valor de initial_equation para currentStep e aleatoriamente pode inverter o lado. Se correct é 0, um valor inválido aleatório é inserido em currentStep.

In [ ]:
# Função de data augmentation para alterar initial equation, incluindo instâncias corretas ou incorretas
def augment_equation(equation, correct):

    # Separa os lados da equação e retira os espaços em branco
    left, right = equation.split('=')
    left = left.strip()
    right = right.strip()

    # Seleciona aleatoriamente uma ou duas operações
    choice = random.choice(["add", "multiply", "div", "add&multiply", "add&div"])

    # Gera constantes aleatórias entre -100 e 100 (inclusive)
    constant_to_add = random.randint(-100, 100)
    constant_to_multiply = random.choice([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 0, -1, -2, -3, -4, -5, -6, -7, -8, -9, -10])
    constant_to_div = random.choice([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, -1, -2, -3, -4, -5, -6, -7, -8, -9, -10])

    # Se correct == 1, adiciona as operações a ambos os lados
    if correct:
      # Adição
      if choice in ["add", "add&multiply", "add&div"]:
        if constant_to_add >= 0:
          left = f"({left}) + {constant_to_add}"
          right = f"({right}) + {constant_to_add}"
        else:
          left = f"({left}) + ({constant_to_add})"
          right = f"({right}) + ({constant_to_add})"

      # Multiplicação
      if choice in ["multiply", "add&multiply"]:
        if constant_to_multiply >= 0:
          left = f"({left}) * {constant_to_multiply}"
          right = f"({right}) * {constant_to_multiply}"
        else:
          left = f"({left}) * ({constant_to_multiply})"
          right = f"({right}) * ({constant_to_multiply})"

      # Divisão
      if choice in ["div", "add&div"]:
        if constant_to_div >= 0:
          left = f"({left}) / {constant_to_div}"
          right = f"({right}) / {constant_to_div}"
        else:
          left = f"({left}) / ({constant_to_div})"
          right = f"({right}) / ({constant_to_div})"

    # Se correct == 0, adiciona as operações apenas no lado esquerdo
    else:

      # Adição
      if choice in ["add", "add&multiply", "add&div"]:
        if constant_to_add >= 0:
          left = f"({left}) + {constant_to_add}"
        else:
          left = f"({left}) + ({constant_to_add})"

      # Multiplicação
      if choice in ["multiply", "add&multiply"]:
        if constant_to_multiply >= 0:
          left = f"({left}) * {constant_to_multiply}"
        else:
          left = f"({left}) * ({constant_to_multiply})"

      # Divisão
      if choice in ["div", "add&div"]:
        if constant_to_div >= 0:
          left = f"({left}) / {constant_to_div}"
        else:
          left = f"({left}) / ({constant_to_div})"

    # Troca os lados da equação aleatoriamente
    if random.choice([True, False]):
        left, right = right, left

    # Retira espaços em branco e retorna equação initial equation modificada
    left = left.replace(" ", "")
    right = right.replace(" ", "")
    return f"{left}={right}"

# Exemplo da equação "2x + 3 = 5" correta e incorreta
equation = "2x + 3 = 5"
augmented_equation1 = augment_equation(equation, 1)
augmented_equation2 = augment_equation(equation, 0)
print("Equação Correta: ", augmented_equation1)
print("Equação Incorreta: ", augmented_equation2)

In [ ]:
# Função de data augmentation para alterar currentStep, copiando o valor de initial equation ou incluindo valores inválidos
def augment_equation_current(equation1, equation2, correct):

    # Seleção de valores aleatórios
    choice = random.choice([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])
    incorretas = random.choice([",", "&", "y", "a", "^", "[", "]"])
    constant1 = random.randint (-100, 100)
    constant2 = random.randint (-100, 100)

    # Se correct == 1, copia initial equation em currentStep e, aleatoriamente, inverte os lados da equação
    if correct:
      if choice in [1, 2, 3, 4, 5, 6]:
        equation2 = equation1
      else:
        equation2 = equation1
        left, right = equation2.split('=')
        left = left.strip()
        right = right.strip()
        left = left.replace(" ", "")
        right = right.replace(" ", "")
        left, right = right, left
        equation2 = f"{left}={right}"

    # Se correct == 0, altera currentStep para um valor inválido aleatório
    else:
      if choice == 1:
        equation2 = "= x"
      elif choice == 2:
        equation2 = "x"
      elif choice == 3:
        equation2 = f"x={constant1}={constant2}"
      elif choice == 4:
        equation2 = f"x={constant1}{incorretas}{constant2}"
      elif choice == 5:
        equation2 = f"{constant1}"
      elif choice == 6:
        equation2 = "x="
      elif choice == 7:
        equation2 = "-"
      elif choice == 8:
        equation2 = "="
      elif choice == 9:
        equation2 = "+"
      elif choice == 10:
        equation2 = "*"
      elif choice == 11:
        equation2 = "/"

    # Retira espaçoes em branco e retorna currentStep atualizado
    equation2 = equation2.replace(" ", "")
    equation2 = equation2.strip()
    return equation2

# Exemplo de uso da função com "2x + 3 = 5" como initial equation e "x = 7" como currentStep, para geração de equação correta e incorreta
equation1 = "2x + 3 = 5"
equation2 = "x = 7"
augmented_equation_current1 = augment_equation_current(equation1, equation2, 1)
augmented_equation_current2 = augment_equation_current(equation1, equation2, 0)
print("Correta: ", augmented_equation_current1)
print("Incorreta: ", augmented_equation_current2)

In [ ]:
# Variável log_temp recebe rótulos corretos (9.360)
log_temp = log[log["step_is_correct"] == 1.0]

# Variável log_temp_subset recebe um subconjunto dos rótulos corretos (8.161)
log_temp_subset = log_temp.head(8161)

# Variável log_temp_subset2 recebe um subconjunto dos rótulos corretos (2.000)
log_temp_subset2 = log_temp.head(2000)

# Variável log_temp_subset3 recebe um subconjunto dos rótulos corretos (1.000)
log_temp_subset3 = log_temp.head(1000)

O código abaixo aplica 5 vezes a função augment_equation com correct == 1, nos rótulos corretos emais uma no subconjunto de 1.000, aumentando de 9.360 instâncias corretas para 57.160.

Após, ele aplica 3 vezes a função augment_equation com correct == 0, nos rótulos corretos e mais uma no subjconjunto de 8.161, aumentando os rótulos incorretos de 20.919 para 57.160.

Por fim, são feitas duas aplicações de augment_equation_current, primeiro com correct == 0 (em 2.000 instâncias) e depois com current == 1 (em 2000 instâncias), aumento os rótulos corretos para 59.160 e os incorretos para 59.160, num total de 118.320 instâncias.

In [ ]:
# Lista de rótulos para as funções
labels = [1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0]
labels_current = [0.0, 1.0]

# Lista para armazenar os DataFrames gerados
new_dataframes = []

# Aplica função augment_equation 5 vezes para geração de equações corretas e 3 para geração de equações incorretas
# Loop para aplicar as funções e criar os DataFrames
for i, lab in enumerate(labels):
    new_rows = log_temp.apply(lambda row: pd.Series({'initial_equation': augment_equation(row['initial_equation'], int(labels[i])),
                                                     'currentStep': row['currentStep'],
                                                     'step_is_correct': labels[i]}), axis=1)
    # Anexar as novas linhas ao DataFrame original
    log = log.append(new_rows, ignore_index=True)

# Aplica função augment_equation para geração de equações incorretas
new_rows = log_temp_subset3.apply(lambda row: pd.Series({'initial_equation': augment_equation(row['initial_equation'], 1),
                                                     'currentStep': row['currentStep'],
                                                     'step_is_correct': 1.0}), axis=1)
# Anexar as novas linhas ao DataFrame original
log = log.append(new_rows, ignore_index=True)

# Aplica função augment_equation para geração de equações incorretas
new_rows = log_temp_subset.apply(lambda row: pd.Series({'initial_equation': augment_equation(row['initial_equation'], 0),
                                                     'currentStep': row['currentStep'],
                                                     'step_is_correct': 0.0}), axis=1)
# Anexar as novas linhas ao DataFrame original
log = log.append(new_rows, ignore_index=True)

for j, lab in enumerate(labels_current):
# Aplica função augment_equation_current 1 vez para geração de equações incorretas e 1 vez para geração de equações corretas
    new_rows = log_temp_subset2.apply(lambda row: pd.Series({'initial_equation': row['initial_equation'],
                                                     'currentStep': augment_equation_current(row['initial_equation'], row['currentStep'], int(labels_current[j])),
                                                     'step_is_correct': labels_current[j]}), axis=1)
    # Anexar as novas linhas ao DataFrame original
    log = log.append(new_rows, ignore_index=True)

# Imprime o log atualizado
log

In [ ]:
# Contar o número de ocorrências de step_is_correct igual a 1 e 0
count_correct = log['step_is_correct'].value_counts()[1.0]
count_incorrect = log['step_is_correct'].value_counts()[0.0]

print("Número de linhas com passos corretos:", count_correct)
print("Número de linhas com passos incorretos:", count_incorrect)

In [ ]:
# Variável counts recebe os valores de rótulos corretos e incorretos
counts = log["step_is_correct"].value_counts()

# Crie um gráfico de barras
plt.bar(counts.index, counts.values, color=['red', 'blue'], edgecolor='k')

# Adicione rótulos e título
plt.xlabel("Status")
plt.ylabel("Instâncias")
plt.title("Contagem de Instâncias com Rótulo Incorreto e Correto")

# Personalize os rótulos no eixo x (opcional)
plt.xticks(counts.index, ['Incorreto', 'Correto'])

# Mostre o gráfico
plt.show()

In [ ]:
# Exclui os valores duplicados novamente, desde que constem como iguais em ambas as colunas: initial equation e currentStep
log = log.drop_duplicates(subset=['initial_equation', 'currentStep'])
log.head(), log.shape

In [ ]:
# Contar o número de ocorrências de step_is_correct igual a 1 e 0
counts = counts = log["step_is_correct"].value_counts()
counts

In [ ]:
# Crie um gráfico de barras
plt.bar(counts.index, counts.values, color=['red', 'blue'], edgecolor='k')

# Adicione rótulos e título
plt.xlabel("Status")
plt.ylabel("Instâncias")
plt.title("Contagem de Instâncias com Rótulo Incorreto e Correto")

# Personalize os rótulos no eixo x (opcional)
plt.xticks(counts.index, ['Incorreto', 'Correto'])

# Mostre o gráfico
plt.show()

In [ ]:
#Imprime log atualizado
log

In [ ]:
#Converte o tipo da coluna step is correct, de float para inteiro.
log['step_is_correct'] = log['step_is_correct'].astype(int)

In [ ]:
log.head()

Embaralhar o conjunto de dados.

In [ ]:
# Cria variável para embaralhar o conjunto de dados
passos_embaralhados = shuffle(log)
passos_embaralhados.head()

In [ ]:
# Embaralha os dados de entrada e os rótulos
X_series = passos_embaralhados[['initial_equation', 'currentStep']]
y_series = passos_embaralhados['step_is_correct']
X = X_series.values.tolist()
y = y_series.tolist()

## Divisão de Dados

Aqui os dados são divididos em três conjuntos: conjunto de treinamento, com 70% das instâncias, conjunto de validação, com 20% das instâncias e conjunto de testes, com 10% das instâncias.

In [ ]:
# Usar o sklearn para dividir os dados
X_train, X_test, y_train, y_test = train_test_split(X_series, y_series, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

## Tokenização, Encoding e Padding

In [ ]:
# Tokenização
max_words = 5000  # Número máximo de palavras a serem mantidas no vocabulário
tokenizer_initial_equation = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer_initial_equation.fit_on_texts(X_train['initial_equation'])

tokenizer_current_step = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer_current_step.fit_on_texts(X_train['currentStep'])

X_train_seq_initial_equation = tokenizer_initial_equation.texts_to_sequences(X_train['initial_equation'])
X_val_seq_initial_equation = tokenizer_initial_equation.texts_to_sequences(X_val['initial_equation'])
X_test_seq_initial_equation = tokenizer_initial_equation.texts_to_sequences(X_test['initial_equation'])

X_train_seq_current_step = tokenizer_current_step.texts_to_sequences(X_train['currentStep'])
X_val_seq_current_step = tokenizer_current_step.texts_to_sequences(X_val['currentStep'])
X_test_seq_current_step = tokenizer_current_step.texts_to_sequences(X_test['currentStep'])

In [ ]:
# Padding
max_sequence_length = 20

X_train_padded_initial_equation = pad_sequences(X_train_seq_initial_equation, maxlen=max_sequence_length, padding='post', truncating='post')
X_val_padded_initial_equation = pad_sequences(X_val_seq_initial_equation, maxlen=max_sequence_length, padding='post', truncating='post')
X_test_padded_initial_equation = pad_sequences(X_test_seq_initial_equation, maxlen=max_sequence_length, padding='post', truncating='post')

X_train_padded_current_step = pad_sequences(X_train_seq_current_step, maxlen=max_sequence_length, padding='post', truncating='post')
X_val_padded_current_step = pad_sequences(X_val_seq_current_step, maxlen=max_sequence_length, padding='post', truncating='post')
X_test_padded_current_step = pad_sequences(X_test_seq_current_step, maxlen=max_sequence_length, padding='post', truncating='post')

# Criação da Rede

In [ ]:
# Construir o modelo GRU
embedding_dim = 128

# Initial Equation
input_initial_equation = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(input_dim=max_words, output_dim=embedding_dim)(input_initial_equation)
gru_layer = GRU(128)(embedding_layer)

#Curreny Step
input_current_step = Input(shape=(max_sequence_length,))
embedding_layer2 = Embedding(input_dim=max_words, output_dim=embedding_dim)(input_current_step)
gru_layer2 = GRU(128)(embedding_layer2)

# Concatena as camadas
concatenated = concatenate([gru_layer, gru_layer2])

#Output Layer
output_layer = Dense(1, activation='sigmoid')(concatenated)
model = Model(inputs=[input_initial_equation, input_current_step], outputs=output_layer)

# Treinamento e Validação

O modelo é treinado e validado com 25 épocas. O melhor modelo é armazenado na variável 'melhor_modelo'. O otimizador utilizado é Adam e a função de loss é Binary Crossentropy.

In [ ]:
# Compilar o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treinar o modelo
checkpoint = ModelCheckpoint('melhor_modelo', monitor='val_loss', save_best_only=True, save_format='tf')
history = model.fit([X_train_padded_initial_equation, X_train_padded_current_step], y_train,
          validation_data=([X_val_padded_initial_equation, X_val_padded_current_step], y_val),
          epochs=25, batch_size=32)

# Imprime as métricas de treinamento e validação
print("Training loss:", history.history['loss'])
print("Training accuracy:", history.history['accuracy'])
print("Validation loss:", history.history['val_loss'])
print("Validation accuracy:", history.history['val_accuracy'])

# Teste
Usando o conjunto de dados de teste para avaliar o modelo

In [ ]:
# Avaliar o modelo no conjunto de teste com return_dict=True
benchmarks = model.evaluate([X_test_padded_initial_equation, X_test_padded_current_step], y_test, return_dict=True)
print(benchmarks)

In [ ]:
N_EPOCHS = 25
# Plotar a curva de aprendizado da perda
plt.figure(figsize=(10, 5))
plt.plot(range(1, N_EPOCHS+1), history.history['loss'], 'r', label='Training Loss')
plt.plot(range(1, N_EPOCHS+1), history.history['val_loss'], 'b', label='Validation Loss')
plt.title('Curva de Aprendizado - Perda')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid()
plt.show()

# Plotar a curva de aprendizado da acurácia
plt.figure(figsize=(10, 5))
plt.plot(range(1, N_EPOCHS+1), history.history['accuracy'], 'r', label='Training Accuracy')
plt.plot(range(1, N_EPOCHS+1), history.history['val_accuracy'], 'b', label='Validation Accuracy')
plt.title('Curva de Aprendizado - Acurácia')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Avaliar o modelo no conjunto de testes
y_pred = model.predict([X_test_padded_initial_equation, X_test_padded_current_step])
y_pred_binary = (y_pred > 0.5).astype(int)

# Calcular a acurácia
accuracy = accuracy_score(y_test, y_pred_binary)
print("Acurácia:", accuracy)

# Crie uma lista de resultados com base na comparação entre Previsão e Rótulo
resultados = ["Certo" if p == r else "Errado" for p, r in zip(y_pred_binary.squeeze(), y_test)]

# Crie um DataFrame com as previsões, probabilidades e entradas originais
results_df = pd.DataFrame({
    "Equação Inicial": X_test['initial_equation'],  # Use os textos decodificados
    "Passo Atual": X_test['currentStep'],  # Use os textos decodificados
    "Probabilidade (%)": y_pred.squeeze(),
    "Previsão": y_pred_binary.squeeze(),
    "Rótulo": y_test ,  # Adicione a lista de rótulos
    "Resultado": resultados  # Adicione a lista de resultados
})

In [ ]:
results_df

In [ ]:
results_df["Resultado"].value_counts()

In [ ]:
# Contagem dos valores "Certo" e "Errado" na coluna "Resultado"
contagem_resultados = results_df["Resultado"].value_counts()

# Calcula o percentual de acerto
percentual_acerto = (contagem_resultados["Certo"] / len(results_df)) * 100

# Imprime o percentual de acerto
print(f"Percentual de acerto: {percentual_acerto:.2f}%")

In [ ]:
# Calcular a matriz de confusão
cm = confusion_matrix(y_test, y_pred_binary.squeeze())

# Definir rótulos das classes (substitua pelos seus rótulos)
class_names = ['Classe 0', 'Classe 1']

# Criar um heatmap da matriz de confusão
plt.figure(figsize=(8, 6))
sns.set(font_scale=1.2)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Rótulos Previstos')
plt.ylabel('Rótulos Reais')
plt.title('Matriz de Confusão')
plt.show()

## Precision, Recall e F1 Score

*    Precision = Verdadeiros Positivos(TP) / (Verdadeiros Positivos(TP) + Falsos Positivos(FP))
*    Recall = Verdadeiros Positivos(TP) / (Verdadeiros Positivos(TP) + Falsos Negativos(FN))
*    F1 Score = 2 * (Precision * Recall) / (Precision + Recall)

In [ ]:
# Exemplo de valores verdadeiros (ground truth) e previsões
verdadeiros = y_test
previsoes = y_pred_binary.squeeze()

# Calcular a precisão e a revocação
precisao = precision_score(verdadeiros, previsoes)
revocacao = recall_score(verdadeiros, previsoes)
# Calcular o F1 Score
f1 = f1_score(verdadeiros, previsoes)

print("Precisão (Precision):", precisao)
print("Revocação (Recall):", revocacao)
print("F1 Score:", f1)

## Curva ROC

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred.squeeze())
roc_auc = auc(fpr, tpr)

# Plote a curva ROC
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='Curva ROC (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taxa de Falsos Positivos')
plt.ylabel('Taxa de Verdadeiros Positivos')
plt.title('Curva ROC')
plt.legend(loc='lower right')
plt.show()